# 🏈 **Análisis del Conjunto de Datos – NFL Big Data Bowl 2026**

---

## 1️⃣ **Objetivo principal del concurso**

El propósito del **NFL Big Data Bowl 2026** es desarrollar modelos predictivos capaces de **estimar la trayectoria futura de los jugadores** (su posición en el campo) **durante el tiempo en que el balón está en el aire**, usando únicamente información disponible **antes del lanzamiento**.

- En términos técnicos:  
  Se trata de un problema de **regresión multivariada y temporal**, donde el modelo debe predecir los valores futuros de **$(x, y)$** para cada jugador y cada instante temporal del vuelo del balón.  

- En términos conceptuales:  
  Se busca capturar cómo los jugadores se mueven según su posición, rol, jugada y contexto (formación, velocidad inicial, etc.), proporcionando información que pueda ayudar a comprender el comportamiento de ofensiva y defensa en tiempo real.

---

## 2️⃣ **Variables de entrada y salida**

### 🔹 **Variables de salida (targets a predecir)**

| Variable | Descripción | Rango aproximado | Tipo |
|-----------|--------------|------------------|------|
| **x** | Posición del jugador a lo largo del eje longitudinal del campo (de una zona de anotación a la otra). | 0 – 120 yardas | Continua |
| **y** | Posición del jugador a lo largo del eje lateral del campo (de una línea lateral a la otra). | 0 – 53.3 yardas | Continua |

> Ambas variables representan la **posición espacial $(x, y)$** del jugador y constituyen el **objetivo de predicción** del modelo.

---

### 🔹 **Variables de entrada (features conocidas antes del lanzamiento)**

Estas variables se utilizan para predecir las posiciones futuras. Se pueden agrupar por categorías:

| Categoría | Variables (ejemplos) | Descripción / utilidad |
|------------|----------------------|------------------------|
| **Identificación y contexto** | `gameId`, `playId`, `frameId`, `nflId`, `team`, `position`, `playType` | Permiten identificar cada jugada, jugador y tipo de acción ofensiva o defensiva. |
| **Estado físico del jugador** | `speed`, `acceleration`, `orientation`, `direction` | Describen el movimiento del jugador justo antes del lanzamiento. |
| **Ubicación actual** | `x_init`, `y_init` | Posición del jugador en el instante previo al lanzamiento (punto de partida de la trayectoria). |
| **Contexto del partido** | `quarter`, `down`, `yardsToGo`, `offenseFormation`, `defensiveTeam` | Variables estratégicas que influyen en la jugada y movimiento. |
| **Condiciones ambientales** | `weather`, `temperature`, `humidity`, `windSpeed`, `fieldType` | Pueden afectar el comportamiento físico y el movimiento. |
| **Relaciones espaciales derivadas** | `dist_to_ball`, `dist_to_QB`, `dist_to_nearest_opponent`, `angle_to_QB` | Variables ingenierizadas que reflejan interacciones jugador-entorno. |

---

## 3️⃣ **Análisis exploratorio de datos (EDA básico)**

Un análisis exploratorio inicial debe orientarse a entender la **distribución espacial, cinemática y contextual** de los jugadores.

| Análisis | Descripción | Ejemplo de visualización / estadística |
|-----------|-------------|-----------------------------------------|
| **Distribución espacial** | Examinar dónde se ubican los jugadores antes del lanzamiento. | Gráficos de dispersión $(x, y)$ por posición (WR, CB, TE). |
| **Velocidad y aceleración** | Analizar distribución de `speed` y `acceleration` para distintos roles. | Boxplots o histogramas por tipo de jugador. |
| **Dirección y orientación** | Observar patrones de movimiento según `direction` u `orientation`. | Diagramas polares o histogramas de ángulos. |
| **Contexto del juego** | Ver frecuencia de `offenseFormation`, `playType`, `yardsToGo`. | Gráficos de barras o heatmaps. |
| **Correlaciones** | Medir relaciones entre variables cinemáticas. | Mapa de correlación entre `speed`, `acceleration`, `orientation`. |
| **Trayectorias** | Representar movimiento real de jugadores durante jugadas. | Animaciones 2D o trayectorias en el campo. |

> El EDA sirve para detectar patrones (por ejemplo, defensas más abiertas en jugadas largas) y evaluar la calidad de los datos antes del modelado.

---
# 🧩 **4️⃣ Procedimientos de codificación de valores faltantes y variables categóricas**

El procesamiento previo de los datos asegura que las trayectorias y atributos de los jugadores sean consistentes y libres de errores antes de entrenar el modelo. A partir del código de referencia, se aplican estrategias específicas de **limpieza**, **imputación** y **codificación**.

---

## 🧩 **Tratamiento de valores faltantes**

### 1️⃣ Variables continuas y físicas
Las variables de movimiento (`x`, `y`, `s`, `a`, `dir`, `o`) se completan de forma temporal:

- Se utiliza **interpolación por jugador**:
  $$
  x_t = x_{t-1} + \frac{(x_{t+1} - x_{t-1})}{2}
  $$
  Esto suaviza los saltos y evita discontinuidades.

- Cuando no existen valores previos o posteriores, se emplean métodos:
  - `bfill` (propagar hacia atrás)
  - `ffill` (propagar hacia adelante)
  - Valores restantes → $0$

### 2️⃣ Variables antropométricas
Se calculan a partir de fórmulas y se imputan por mediana:

| Variable | Descripción | Método |
|-----------|--------------|--------|
| `player_height` | Conversión pies-pulgadas a pulgadas totales | $\text{inches} = \text{feet} \times 12 + \text{inches}$ |
| `player_age` | Edad a partir de fecha de nacimiento | $\text{edad} = \frac{\text{fecha}_{ref} - \text{fecha}_{nac}}{365.25}$ |
| `bmi` | Índice de masa corporal | $BMI = \frac{\text{peso} \times 703}{(\text{altura en pulgadas})^2}$ |

Si alguno de estos datos está ausente, se imputa con la **mediana** de la variable correspondiente.

---

## 🧾 **Codificación de variables categóricas**

La función `encode_categorical()` emplea **Label Encoding** sobre tres columnas clave:

| Variable | Descripción | Codificación |
|-----------|--------------|--------------|
| `player_position` | Posición del jugador (WR, CB, QB...) | Asignación de un entero |
| `player_side` | Lado del campo (left/right) | Asignación binaria |
| `player_role` | Rol (ofensivo/defensivo) | Entero según rol |

El proceso garantiza que cada categoría se convierta en un número y los *encoders* se guardan para mantener coherencia en el conjunto de prueba.

---

# ⚙️ **5️⃣ Estrategia de ingeniería de características (Feature Engineering)**

El modelo requiere capturar **dinámicas físicas, espaciales y temporales**.  
El código implementa una función `engineer_features()` que crea nuevas variables a partir de las originales.

---

## 🧮 **A. Componentes de velocidad y orientación**

Descomposición vectorial de la velocidad según la dirección:

$$
v_x = s \cdot \cos(\theta), \quad v_y = s \cdot \sin(\theta)
$$

Descomposición del ángulo de orientación:

$$
o_x = \cos(o), \quad o_y = \sin(o)
$$

Estas variables permiten modelar la cinemática del movimiento.

---

## 🏈 **B. Relación con el balón**

Si existen las coordenadas del punto donde aterriza el balón:

- **Distancia al balón:**
  $$
  d_{ball} = \sqrt{(x - x_b)^2 + (y - y_b)^2}
  $$

- **Ángulo hacia el balón:**
  $$
  \alpha_{ball} = \arctan2(y_b - y, x_b - x)
  $$

- **Velocidad proyectada hacia el balón:**
  $$
  v_{→ball} = s \cdot \cos(\theta - \alpha_{ball})
  $$

Estas relaciones capturan cómo el jugador se desplaza respecto al punto objetivo.

---

## 🧭 **C. Posición dentro del campo**

| Variable | Definición |
|-----------|------------|
| `dist_to_left_sideline` | $y$ |
| `dist_to_right_sideline` | $53.3 - y$ |
| `dist_to_nearest_sideline` | $\min(y, 53.3 - y)$ |
| `dist_to_endzone` | $120 - x$ |

Estas distancias delimitan el espacio útil de movimiento del jugador.

---

## ⏱️ **D. Variables temporales (lags y cambios)**

Se generan variables de *retardo temporal*:

$$
x_{lag1} = x(t-1), \quad s_{lag2} = s(t-2)
$$

Y se calculan los cambios entre frames consecutivos:

$$
\Delta s = s_t - s_{t-1}, \quad \Delta a = a_t - a_{t-1}
$$

La variación angular se ajusta para mantener continuidad (valores entre $-180°$ y $180°$).

---

## 📊 **E. Estadísticas móviles**

Se aplican promedios y desviaciones estándar móviles sobre ventanas de 3 frames:

$$
s_{roll\_mean} = \frac{1}{3}\sum_{i=t-2}^{t}s_i, \quad a_{roll\_std} = \sqrt{\frac{1}{3}\sum_{i=t-2}^{t}(a_i - \bar{a})^2}
$$

Estas variables suavizan ruidos y capturan tendencias locales.

---

## 🧠 **F. Atributos estáticos del jugador**

Se integran variables personales que afectan el desempeño:

- Altura (`height_inches`)
- Peso (`player_weight`)
- Edad (`player_age`)
- Índice de masa corporal (`bmi`)
- Rol y posición (`player_position`, `player_role`)

Estas variables permiten al modelo diferenciar comportamientos según el perfil físico y táctico.

---

# 🧩 **6️⃣ Detalles para comprender, interpretar y analizar el conjunto de datos**

El *pipeline* organiza y normaliza los datos en tres fases críticas:

---

## ⚙️ **A. Normalización de la dirección de juego**

Todas las jugadas se orientan en la misma dirección.  
Si la jugada va hacia la izquierda:

$$
x' = 120 - x, \quad y' = 53.3 - y
$$

Esto asegura que el modelo no tenga que aprender dos orientaciones distintas.

---

## 📏 **B. Estandarización numérica**

Se aplica `StandardScaler()`:

$$
z = \frac{x - \mu}{\sigma}
$$

A todas las variables continuas, mejorando la estabilidad numérica y la convergencia durante el entrenamiento.

---

## ⏳ **C. Secuencias temporales**

Cada jugador se representa mediante una **ventana temporal** de longitud $L = 10$ frames:

$$
\text{Secuencia}_i = [x_{t-L+1}, ..., x_t]
$$

El modelo LSTM procesa estas secuencias para capturar la evolución temporal de las trayectorias.

---

## 📊 **D. Evaluación de desempeño**

Las métricas principales son:

- **Error cuadrático medio (RMSE):**
  $$
  RMSE = \sqrt{\frac{1}{N}\sum_i ((\hat{x}_i - x_i)^2 + (\hat{y}_i - y_i)^2)}
  $$

- **Error absoluto medio (MAE):**
  $$
  MAE = \frac{1}{N}\sum_i (|\hat{x}_i - x_i| + |\hat{y}_i - y_i|)
  $$

- **Distancia euclidiana promedio:**
  $$
  d_{mean} = \frac{1}{N}\sum_i \sqrt{(\hat{x}_i - x_i)^2 + (\hat{y}_i - y_i)^2}
  $$

---

## 🎯 **E. Interpretación visual**

El código genera gráficos de dispersión $(x_{true}, x_{pred})$ y $(y_{true}, y_{pred})$, histogramas de error y curvas acumulativas de precisión para verificar qué porcentaje de predicciones cae dentro de 5, 10 o 15 yardas.

---

## 🔍 **F. Síntesis final**

El modelo LSTM integra:

- Entradas dinámicas (trayectorias y velocidades)  
- Entradas estáticas (atributos y contexto)  
- Predicción de salida multivariada $(x, y)$  

🧠 Este flujo reproduce fielmente el comportamiento espacial y temporal de los jugadores durante las jugadas, combinando **física, geometría y aprendizaje secuencial** en un entorno normalizado y escalable.

✅ **Explicación corta del proceso de preprocesamiento (NFL Big Data Bowl 2026)**

Este código construye el **dataset de entrenamiento limpio y estandarizado** para los modelos de predicción del concurso NFL Big Data Bowl 2026.

1. **Unificación de datos:** carga y concatena automáticamente todos los archivos semanales (`input_2023_w*.csv`) en un solo DataFrame.
2. **Normalización espacial:** corrige las coordenadas de jugadas que van hacia la izquierda para que todas las jugadas estén orientadas en la misma dirección del campo.
3. **Ingeniería de características:** genera variables adicionales como velocidad en eje X/Y, orientación y distancias a líneas del campo.
4. **Creación de etiquetas (`x_target`, `y_target`):** define las posiciones futuras del jugador desplazadas `H` frames adelante, necesarias para el aprendizaje supervisado.
5. **Eliminación de fugas de información:** remueve variables que podrían anticipar el resultado real del modelo.
6. **Exportación:** guarda el resultado final como `train_ready.csv`, un dataset limpio, sin fugas y con variables numéricas listas para el entrenamiento.

➡️ En resumen, este script automatiza todo el flujo de **preprocesamiento de datos crudos a datos modelables**, garantizando coherencia, integridad y compatibilidad con los modelos de predicción posteriores.


In [ ]:
# ======================================================
# 🏈 NFL Big Data Bowl 2026 - Preprocesamiento Completo
# Descripción:
# Este script unifica los archivos semanales, crea etiquetas
# (x_target, y_target), elimina fugas de información y genera
# un dataset limpio y listo para entrenamiento (train_ready.csv)
# ======================================================

import pandas as pd
import numpy as np
from glob import glob
import os

# ======================================================
# 1️⃣ CONFIGURACIÓN INICIAL
# ======================================================

# 📂 Carpeta donde están los archivos semanales
DATA_PATH = "/content"   # <-- ajusta si tus archivos están en otra carpeta

# ⏱️ Horizonte temporal (número de frames hacia adelante a predecir)
H = 5  # puedes ajustar según la especificación del reto o tus pruebas

# ======================================================
# 2️⃣ FUNCIONES MODULARES
# ======================================================

def load_weekly_data(path_pattern="input_2023_w*.csv"):
    """Carga y concatena todos los archivos semanales."""
    files = sorted(glob(os.path.join(DATA_PATH, path_pattern)))
    if not files:
        raise FileNotFoundError("❌ No se encontraron archivos semanales. Verifica la ruta.")
    dfs = [pd.read_csv(f) for f in files]
    data = pd.concat(dfs, ignore_index=True)
    print(f"✅ Se cargaron {len(files)} archivos semanales ({data.shape[0]} filas).")
    return data

def normalize_direction(df):
    """Normaliza coordenadas de jugadas que van hacia la izquierda."""
    df = df.copy()
    left_mask = df["play_direction"] == "left"
    df.loc[left_mask, "x"] = 120 - df.loc[left_mask, "x"]
    df.loc[left_mask, "y"] = 53.3 - df.loc[left_mask, "y"]
    df.loc[left_mask, "dir"] = (df.loc[left_mask, "dir"] + 180) % 360
    df.loc[left_mask, "o"] = (df.loc[left_mask, "o"] + 180) % 360
    return df

def create_targets(df, horizon=5):
    """Crea etiquetas x_target e y_target desplazadas hacia el futuro."""
    df = df.sort_values(["game_id", "play_id", "nfl_id", "frame_id"]).copy()
    df["x_target"] = df.groupby(["game_id", "play_id", "nfl_id"])["x"].shift(-horizon)
    df["y_target"] = df.groupby(["game_id", "play_id", "nfl_id"])["y"].shift(-horizon)
    df = df.dropna(subset=["x_target", "y_target"])
    print(f"🎯 Targets creados a horizonte h={horizon} frames.")
    return df

def engineer_features(df):
    """Crea características adicionales de movimiento."""
    df = df.copy()
    df["vx"] = df["s"] * np.cos(np.radians(df["dir"]))
    df["vy"] = df["s"] * np.sin(np.radians(df["dir"]))
    df["ox"] = np.cos(np.radians(df["o"]))
    df["oy"] = np.sin(np.radians(df["o"]))
    df["dist_to_left_sideline"] = df["y"]
    df["dist_to_right_sideline"] = 53.3 - df["y"]
    df["dist_to_nearest_sideline"] = np.minimum(df["y"], 53.3 - df["y"])
    df["dist_to_endzone"] = 120 - df["x"]
    return df

def remove_leakage(df):
    """Elimina columnas con fuga de información."""
    cols_leak = ["dist_to_ball", "angle_to_ball", "vel_toward_ball"]
    df = df.drop(columns=[c for c in cols_leak if c in df.columns], errors="ignore")
    print("🧹 Eliminadas columnas con fuga de información.")
    return df

# ======================================================
# 3️⃣ PIPELINE PRINCIPAL
# ======================================================

def build_training_dataset(h=H):
    """Ejecuta todo el pipeline y genera train_ready.csv."""
    print("🚀 Iniciando construcción del dataset...")

    # 1. Cargar datos
    df = load_weekly_data("input_2023_w*.csv")

    # 2. Filtrar solo jugadores a predecir
    df = df[df["player_to_predict"] == 1].copy()
    print(f"📌 Jugadores a predecir: {df.shape[0]} filas.")

    # 3. Normalizar dirección
    df = normalize_direction(df)

    # 4. Crear características básicas
    df = engineer_features(df)

    # 5. Crear etiquetas (targets)
    df = create_targets(df, horizon=h)

    # 6. Eliminar fugas
    df = remove_leakage(df)

    # 7. Guardar dataset final
    out_path = os.path.join(DATA_PATH, "train_ready.csv")
    df.to_csv(out_path, index=False)
    print(f"✅ Dataset final guardado: {out_path}")
    print(f"🧾 Filas: {df.shape[0]} | Columnas: {df.shape[1]}")
    return df

# ======================================================
# 4️⃣ EJECUCIÓN DIRECTA
# ======================================================

# Ejecuta todo el flujo automáticamente
df_train = build_training_dataset(H)

# Vista previa
df_train.head()


🚀 Iniciando construcción del dataset...
✅ Se cargaron 18 archivos semanales (4880579 filas).
📌 Jugadores a predecir: 1303440 filas.
🎯 Targets creados a horizonte h=5 frames.
🧹 Eliminadas columnas con fuga de información.
✅ Dataset final guardado: /content/train_ready.csv
🧾 Filas: 1073215 | Columnas: 33


,game_id,play_id,player_to_predict,nfl_id,frame_id,play_direction,absolute_yardline_number,player_name,player_height,player_weight,...,vx,vy,ox,oy,dist_to_left_sideline,dist_to_right_sideline,dist_to_nearest_sideline,dist_to_endzone,x_target,y_target
208,2023090700,101,True,44930,1,right,42,Josh Reynolds,6-3,196,...,-0.000000,0.000000,0.156952,0.987606,12.17,41.13,12.17,78.97,41.20,12.26
209,2023090700,101,True,44930,2,right,42,Josh Reynolds,6-3,196,...,-0.000000,0.000000,0.134678,0.990889,12.17,41.13,12.17,78.97,41.33,12.33
210,2023090700,101,True,44930,3,right,42,Josh Reynolds,6-3,196,...,0.008443,0.018131,0.116151,0.993232,12.18,41.12,12.18,78.95,41.54,12.42
211,2023090700,101,True,44930,4,right,42,Josh Reynolds,6-3,196,...,0.100498,0.149332,0.099493,0.995038,12.20,41.10,12.20,78.93,41.79,12.53
212,2023090700,101,True,44930,5,right,42,Josh Reynolds,6-3,196,...,0.290068,0.490674,0.031236,0.999512,12.22,41.08,12.22,78.89,42.12,12.68


✅ **Explicación corta del proceso de preprocesamiento final (Dataset listo para modelado)**

Este código toma el dataset intermedio `train_ready.csv` y lo transforma en un **dataset completamente numérico y estandarizado**, preparado para entrenar modelos de Machine Learning.

1. **Conversión de variables físicas:** transforma la altura de formato `'6-3'` a pulgadas y calcula la edad del jugador a partir de su fecha de nacimiento.
2. **Eliminación de columnas no numéricas:** se eliminan las columnas textuales originales (`player_height`, `player_birth_date`) que no aportan directamente al modelado.
3. **Codificación de variables categóricas:** las columnas como `player_position`, `player_side` y `player_role` se convierten en valores numéricos mediante `LabelEncoder`.
4. **Estandarización de variables numéricas:** se aplica `StandardScaler` para normalizar las variables continuas y evitar que las diferencias de escala afecten el aprendizaje de los modelos.
5. **Protección de variables clave:** se excluyen de la normalización los identificadores (`game_id`, `play_id`, etc.) y las variables objetivo (`x_target`, `y_target`).
6. **Exportación final:** se guarda el dataset como `train_ready_final.csv`, con todas las columnas listas para ser usadas por los modelos predictivos.

➡️ En resumen, este script realiza la **última etapa del preprocesamiento**, garantizando que todos los datos estén **limpios, numéricos y en la misma escala**, optimizando así la calidad y estabilidad del entrenamiento de los modelos.


In [ ]:
# ======================================================
# 🏈 PREPROCESAMIENTO FINAL DEL DATASET - LISTO PARA MODELADO
# Descripción:
# Este script toma el archivo train_ready.csv y lo transforma
# completamente en un dataset 100% numérico, codificado y escalado,
# listo para entrenar los 10 modelos de Machine Learning.
# ======================================================

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
import os

# ======================================================
# 1️⃣ CONFIGURACIÓN
# ======================================================

DATA_PATH = "/content"       # Cambia si tus archivos están en otra ruta
INPUT_FILE = "/content/train_ready.csv"
OUTPUT_FILE = "train_ready_final.csv"

# ======================================================
# 2️⃣ FUNCIONES AUXILIARES
# ======================================================

def parse_height(height_str):
    """Convierte una altura tipo '6-3' a pulgadas (ej: 75)."""
    if pd.isna(height_str):
        return np.nan
    try:
        feet, inches = map(int, str(height_str).split('-'))
        return feet * 12 + inches
    except:
        return np.nan

def calculate_age(birth_date, ref_date='2023-09-01'):
    """Convierte la fecha de nacimiento en edad en años."""
    try:
        birth = pd.to_datetime(birth_date, errors='coerce')
        ref = pd.to_datetime(ref_date)
        return (ref - birth).dt.days / 365.25
    except:
        return np.nan

def encode_categoricals(df, categorical_cols):
    """Codifica variables categóricas usando LabelEncoder."""
    df = df.copy()
    for col in categorical_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
    return df

# ======================================================
# 3️⃣ CARGA DEL DATASET
# ======================================================

df = pd.read_csv(os.path.join(DATA_PATH, INPUT_FILE))
print(f"✅ Dataset cargado: {df.shape[0]} filas, {df.shape[1]} columnas")

# ======================================================
# 4️⃣ LIMPIEZA Y TRANSFORMACIÓN
# ======================================================

# Conversión de altura y edad
df["height_inches"] = df["player_height"].apply(parse_height)
df["player_age"] = calculate_age(df["player_birth_date"])

# Eliminar columnas originales no numéricas
cols_drop = ["player_height", "player_birth_date"]
df = df.drop(columns=[c for c in cols_drop if c in df.columns], errors="ignore")

# Codificación de categóricas
categorical_cols = [c for c in ["player_position", "player_side", "player_role"]
                    if c in df.columns]
df = encode_categoricals(df, categorical_cols)
print("🔢 Variables categóricas codificadas numéricamente.")

# ======================================================
# 5️⃣ ESCALADO DE VARIABLES NUMÉRICAS
# ======================================================

# Seleccionar columnas numéricas
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Evitar escalar los identificadores ni las variables objetivo
cols_to_exclude = ["game_id", "play_id", "nfl_id", "frame_id",
                   "x_target", "y_target"]
scale_cols = [c for c in num_cols if c not in cols_to_exclude]

scaler = StandardScaler()
df[scale_cols] = scaler.fit_transform(df[scale_cols])
print(f"⚙️ Escaladas {len(scale_cols)} columnas numéricas.")

# ======================================================
# 6️⃣ GUARDADO FINAL
# ======================================================

output_path = os.path.join(DATA_PATH, OUTPUT_FILE)
df.to_csv(output_path, index=False)

print("\n✅ Dataset final guardado correctamente.")
print(f"📁 Ruta: {output_path}")
print(f"🧾 Filas: {df.shape[0]} | Columnas: {df.shape[1]}")
print("🎯 Listo para entrenar los 10 modelos del punto 2.")


✅ Dataset cargado: 1073215 filas, 33 columnas
🔢 Variables categóricas codificadas numéricamente.
⚙️ Escaladas 24 columnas numéricas.

✅ Dataset final guardado correctamente.
📁 Ruta: /content/train_ready_final.csv
🧾 Filas: 1073215 | Columnas: 33
🎯 Listo para entrenar los 10 modelos del punto 2.


✅ **Explicación corta del proceso de limpieza final del dataset numérico**

Este código realiza la **última depuración del dataset** antes del entrenamiento de los modelos, asegurando que **todas las columnas sean numéricas** y eliminando cualquier texto residual.

1. **Carga del dataset:** se lee el archivo `train_ready_final.csv` previamente procesado.
2. **Detección de columnas no numéricas:** se identifican variables de tipo texto o categóricas que aún no fueron convertidas.
3. **Eliminación de texto irrelevante:** se eliminan columnas descriptivas como `player_name` y `play_direction`, ya que no aportan información útil para el modelado.
4. **Verificación de tipos:** se revisa que todas las columnas restantes sean numéricas o booleanas.
5. **Guardado del dataset limpio:** se exporta como `train_ready_final_numeric.csv`, garantizando que el conjunto esté **100 % numérico, sin fugas de información y completamente listo para el entrenamiento** de modelos de Machine Learning.


In [ ]:
import pandas as pd

# Cargar el dataset final
df = pd.read_csv("/content/train_ready_final.csv")

# Verificar columnas no numéricas
non_numeric = df.select_dtypes(exclude=["number", "bool"]).columns.tolist()
print("Columnas no numéricas detectadas:", non_numeric)

# Eliminar columnas irrelevantes de texto
cols_to_drop = ["player_name", "play_direction"]  # Ajusta si cambia el nombre
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors="ignore")

# Confirmar limpieza
print("Tipos de datos finales:")
print(df.dtypes.value_counts())

# Guardar dataset final limpio
df.to_csv("/content/train_ready_final_numeric.csv", index=False)
print("✅ Archivo guardado: train_ready_final_numeric.csv — 100 % numérico y listo.")


Columnas no numéricas detectadas: ['play_direction', 'player_name']
Tipos de datos finales:
float64    26
int64       4
bool        1
Name: count, dtype: int64
✅ Archivo guardado: train_ready_final_numeric.csv — 100 % numérico y listo.
